In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

# Parameters
n_epochs = 100
batch_size = 128
hidden_size = 128
seq_length = 70
max_seq_length = 110 # I have too many data points and not enough RAM so filtering out sequences that are too long

In [36]:
# Parameters: max length
filename = "all_arxiv_titles.txt"
raw_text = open(filename, 'r', encoding='utf-8').read().lower()
chars = sorted(list(set(raw_text)))

raw_text = raw_text.splitlines()

titles = []

def special_characters(s):
  special_characters = ["\\", "^", "!", "*", "/"]
  for c in special_characters:
    if c in s:
      return True
    return False

# Cleaning up data: Filtering out titles with special characters for easier learning and because I have too many datapoints
for line in raw_text:
  if len(line) > seq_length and len(line) < max_seq_length and not special_characters(line):
    titles.append(line)

In [ ]:
#Data visualization: See the longest
print("We have " + str(len(titles)) + " in total")
lengths = [len(s) for s in titles]
max_index = np.argmax(lengths)
maxlen, minlen = max(lengths), min(lengths)
print(titles[max_index], max_index, len(titles[max_index]))

#vocab size
num_chars = len(chars)

total variation distance between sdes with stable noise and brownian motion with applications to poisson pdes 39 109


In [ ]:
# Constructing Dataset Substrings
char_to_int = dict((c,i) for i,c in enumerate(chars))

dataX = []
dataY = []

# The input is a bunch of substrings of each title at a fixed length
# The output/label is the next character
for title in titles:
  for i in range(0, len(title) - seq_length):
      dataX.append(title[i: i + seq_length])
      dataY.append(title[i + seq_length])


In [ ]:
# Dataset Preparation: Constructing the tensors

# First dimension is the number of datapoints
# Second dimension: each datapoint has seq_length timesteps i.e. characters
# Third dimension: Each character is one hot encoded, so it's a vector of len(chars)
x = np.zeros((len(dataX), seq_length, len(chars)), dtype=float)
y = np.zeros((len(dataX), len(chars)), dtype=float)

for i, title_prefix in enumerate(dataX): #i-th data point, i.e. i-th title
    for t, char in enumerate(title_prefix): #tth character in the title
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[dataY[i]]] = 1

x = torch.tensor(x).to(torch.float32)
y = torch.tensor(y).to(torch.float32)

In [ ]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Create DataLoaders
train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
test_loader = data.DataLoader(data.TensorDataset(X_test, y_test), shuffle=False, batch_size=batch_size)


In [ ]:
print(X_train.shape)

torch.Size([69194, 70, 62])


In [37]:
class GRUModel(nn.Module):
    def __init__(self, hidden_size, maxlen, num_chars):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size=num_chars, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_chars)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [61]:
# Initialize model, optimizer, and loss function
model = GRUModel(hidden_size, seq_length, num_chars)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")

pretrained = True
if pretrained:
  try:
      checkpoint = torch.load("gru.pth")
      print(checkpoint)
      model.load_state_dict(checkpoint[0])
      char_to_int = checkpoint[1]
      print("Loaded pre-trained model")
  except FileNotFoundError:
      print("No pre-trained model found, training from scratch")


[OrderedDict([('gru.weight_ih_l0', tensor([[-0.0775, -0.0241, -0.0691,  ..., -0.0652, -0.0764, -0.0978],
        [-0.0634, -0.7764,  0.0173,  ..., -1.8852,  0.0607,  0.4673],
        [ 0.0684, -0.7741,  0.0130,  ...,  0.7599,  0.0452, -0.2084],
        ...,
        [ 0.0553,  0.4371,  0.0355,  ...,  1.0812,  0.0123,  0.4369],
        [ 0.0798,  0.0936, -0.0465,  ..., -0.8510, -0.0583, -0.4165],
        [-0.0590,  0.0244, -0.0711,  ...,  0.2479, -0.0270, -0.2188]])), ('gru.weight_hh_l0', tensor([[-1.0715, -0.4643,  0.2535,  ..., -0.2104,  0.0946, -0.2188],
        [-0.1557, -0.5505, -0.1002,  ...,  0.1397, -0.3605,  0.0515],
        [-0.3736, -0.0759, -0.1117,  ..., -0.1969, -0.0260,  0.0476],
        ...,
        [ 0.1859,  0.2638,  0.4203,  ..., -0.2255,  0.5151, -0.2171],
        [ 0.0764, -0.2539,  0.3958,  ...,  0.3364,  0.1704,  0.1478],
        [-0.3593,  0.2668, -0.0479,  ..., -0.0795, -0.3739,  0.0870]])), ('gru.bias_ih_l0', tensor([ 2.7191e-01, -1.3049e-03, -1.7128e-01,  2.132

In [ ]:
best_model = None
best_loss = np.inf

# Training loop
for epoch in tqdm(range(n_epochs)):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        # X_batch = (X_batch.reshape(batch_size, seq_length, -1))
        # print(X_batch.shape, batch_size)
        y_pred = model(X_batch) #forward pass
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            test_loss += loss.item()

    test_loss /= len(test_loader.dataset)

    if test_loss < best_loss:
        best_loss = test_loss
        best_model = model.state_dict()

    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

# Save the best model and char_to_int dictionary
torch.save([best_model, char_to_int], "single-char_finetune.pth")


max leng 70


  1%|          | 1/100 [01:25<2:20:42, 85.28s/it]

Epoch 1: Train Loss: 2.7461, Test Loss: 2.4225


  2%|▏         | 2/100 [02:55<2:24:08, 88.25s/it]

Epoch 2: Train Loss: 2.2935, Test Loss: 2.1983


...

Epoch 98: Train Loss: 0.2263, Test Loss: 0.9470


 99%|█████████▉| 99/100 [2:22:19<01:24, 84.67s/it]

Epoch 99: Train Loss: 0.2155, Test Loss: 0.9638


100%|██████████| 100/100 [2:23:40<00:00, 86.21s/it]

Epoch 100: Train Loss: 0.2210, Test Loss: 0.9381


In [ ]:
torch.save([best_model, char_to_int], "single-char_finetune.pth")


In [79]:
def generate_sequence(model, seed, n_chars, char_to_int, int_to_char):
    model.eval()
    with torch.no_grad():
        x = np.zeros((1, len(seed), len(char_to_int)), dtype=float)
        for t, char in enumerate(seed):
            x[0, t, char_to_int[char]] = 1.0
        x = torch.tensor(x, dtype=torch.float32)

        hidden = None
        generated_text = seed

        for _ in range(n_chars):
            output = model(x)
            output = torch.softmax(output, dim=1)
            char_index = torch.multinomial(output, 1).item()
            char = int_to_char[char_index]
            generated_text += char

            # Prepare input for next character generation
            x = np.zeros((1, 1, len(char_to_int)), dtype=float)
            x[0, 0, char_index] = 1.0
            x = torch.tensor(x, dtype=torch.float32)

    return generated_text


int_to_char = dict((i,c) for i,c in enumerate(chars))

# Generate a sequence
seed = 'Parametri'.lower()
x = np.zeros((1, len(seed), len(chars)), dtype=float)
for t, char in enumerate(seed): #tth character in the title
     x[0, t, char_to_int[char]] = 1.0
x = torch.tensor(x, dtype=torch.float32)

# print(x)
n_chars = 80  # Number of characters to generate
generated_text = generate_sequence(model, seed, n_chars, char_to_int, int_to_char)
print(generated_text)


parametrich ph jed x p ph dinin k7 win pedininin d dy wininin dininin ph-n ph pth 2 majed
